In [1]:
##предварительная обработка текста

In [1]:
import numpy as np #матрицы
import pandas as pd #датафреймы, таблицы
import re
import nltk
import string

In [4]:
#!pip install numpy

In [2]:
##Чтение датасета с текстом
df = pd.read_csv("reviews.csv", encoding='UTF8', sep="\t")
df.head(10)

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
5,Очень глубокие проймы,negative
6,Я недовольна заказом.Я вот одного не понимаю п...,negative
7,"заказала размер s на от 64,об 94,начнем с того...",negative
8,Заказ я сделала в июле. С тех пор посылка отсл...,negative
9,Ужасное качество товара!,negative


In [44]:
df.iloc[3][0]

'товар не пришел, продавец продлил защиту без моего согласия. от продавца одни обещания'

In [3]:
##Нижний регистр. Например, Товар --> товар, УРА --> ура
df['review']  = df['review'].str.lower()
df['review'].head()

0    качество плохое пошив ужасный (горловина напер...
1    товар отдали другому человеку, я не получила п...
2    ужасная синтетика! тонкая, ничего общего с пре...
3    товар не пришел, продавец продлил защиту без м...
4        кофточка голая синтетика, носить не возможно.
Name: review, dtype: object

In [28]:
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"товар отдали другому человеку, получила посылк...",negative
2,"ужасная синтетика! тонкая, общего представленн...",negative
3,"товар пришел, продавец продлил защиту моего со...",negative
4,"кофточка голая синтетика, носить возможно.",negative


In [18]:
#Удаление пунктуации
df['review'] = df['review'].str.replace('[^\w\s]','')
df['review'].head()

0    качество плохое пошив ужасный (горловина напер...
1    товар отдали другому человеку, получила посылк...
2    ужасная синтетика! тонкая, общего представленн...
3    товар пришел, продавец продлил защиту моего со...
4           кофточка голая синтетика, носить возможно.
Name: review, dtype: object

In [29]:
?word

Object `word` not found.


In [32]:
#предыдущее удаление пунктуации почему-то не сработало, поэтому вот
#к сожалению, это тоже перестало работать
exclude = set(string.punctuation)
for i in range(0,len(df)):
    df.iloc[i][0] = ''.join(symbol for symbol in df.iloc[i][0] if symbol not in exclude)
#s = ''.join(ch for ch in s if ch not in exclude)
df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_2592\1742576963.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i][0] = ''.join(symbol for symbol in df.iloc[i][0] if symbol not in exclude)


,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"товар отдали другому человеку, получила посылк...",negative
2,"ужасная синтетика! тонкая, общего представленн...",negative
3,"товар пришел, продавец продлил защиту моего со...",negative
4,"кофточка голая синтетика, носить возможно.",negative


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
#Удаление стоп-слов
#Импорт стоп-слов из бибилиотеки nltk
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('russian'))
# функция, удаляющая стопслова из текстов

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
# Применим функцию удаления стоп-слов
df["review"] = df["review"].apply(stopwords)
df["review"].head()

0    качество плохое пошив ужасный (горловина напер...
1    товар отдали другому человеку, получила посылк...
2    ужасная синтетика! тонкая, общего представленн...
3    товар пришел, продавец продлил защиту моего со...
4           кофточка голая синтетика, носить возможно.
Name: review, dtype: object

In [52]:
#Удаление высокочастотных слов
from collections import Counter
cnt = Counter()
for text in df["review"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('не', 80205),
 ('на', 53139),
 ('и', 42323),
 ('в', 32788),
 ('очень', 27788),
 ('но', 21844),
 ('размер', 19024),
 ('что', 14570),
 ('с', 13687),
 ('как', 13376)]

In [53]:
#ужаление часто встречающиеся слов в корпусе при помощи tf-idf
# Удаление самых частотных слов
freq = set([w for (w, wc) in cnt.most_common(10)])
# функция удаления слов
def freqwords(text):
    return " ".join([word for word in str(text).split() if word not 
in freq])
# применение функции
df["review"] = df["review"].apply(freqwords)
df["review"].head()

0    качество плохое пошив ужасный горловина напере...
1    товар отдали другому человеку я получила посыл...
2    ужасная синтетика тонкая ничего общего предста...
3    товар пришел продавец продлил защиту без моего...
4             кофточка голая синтетика носить возможно
Name: review, dtype: object

In [54]:
#Удаление низкочастотных слов
freq = pd.Series(' '.join(df['review']).split()).value_counts()[-10:] # 10 rare words
freq = list(freq.index)
df['review'] = df['review'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['review'].head()

0    качество плохое пошив ужасный горловина напере...
1    товар отдали другому человеку я получила посыл...
2    ужасная синтетика тонкая ничего общего предста...
3    товар пришел продавец продлил защиту без моего...
4             кофточка голая синтетика носить возможно
Name: review, dtype: object

In [55]:
# Удаление эмодзи
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
#passing the emoji function to 'text_rare'
df['review'] = df['review'].apply(emoji)

In [56]:
#Удаление цифр
df['review'] = df['review'].str.replace('\d+', '') 
df['review'].head()

0    качество плохое пошив ужасный горловина напере...
1    товар отдали другому человеку я получила посыл...
2    ужасная синтетика тонкая ничего общего предста...
3    товар пришел продавец продлил защиту без моего...
4             кофточка голая синтетика носить возможно
Name: review, dtype: object

In [57]:
#Удаление URL
# Function for url's
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
# Examples
text = "This is my website, https://www.link.com"
remove_urls(text)
#Passing the function to 'text_rare'
df['review'] = df['review'].apply(remove_urls)

In [58]:
#Удаление HTML-тегов
from bs4 import BeautifulSoup
#Function for removing html
def html(text):
    return BeautifulSoup(text, "lxml").text
# Examples
text = """<div>
<h1> Это </h1>
<p> странца</p>
<a href="https://www.link.com/"> пример</a>
</div>
"""
print(html(text))
# Passing the function to 'review'
df['review'] = df['review'].apply(html)


 Это 
 странца
 пример




In [59]:
#Токенизация
def tokenization(text):
    text = re.split('\W+', text)
    return text
# Passing the function to 'text_rare' and store into'text_token'
df['review'] = df['review'].apply(lambda x: tokenization(x.lower()))
df[['review']].head()

,review
0,"[качество, плохое, пошив, ужасный, горловина, ..."
1,"[товар, отдали, другому, человеку, я, получила..."
2,"[ужасная, синтетика, тонкая, ничего, общего, п..."
3,"[товар, пришел, продавец, продлил, защиту, без..."
4,"[кофточка, голая, синтетика, носить, возможно]"


In [36]:
!pip install pymorphy2

In [40]:
#Можно выполнить при помощи NLTK. Заодно проведем лемматизацию
from pymorphy2 import MorphAnalyzer #вот тут тоже ничего не получилось
from nltk import sent_tokenize, word_tokenize, regexp_tokenize

def tokenize_lemmas(sent, pat=r"(?u)\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(sent, pat)]
df["review"] = df["review"].map(lambda x: " ".join(tokenize_lemmas(x)))
df[['review']].head()

AttributeError: module 'inspect' has no attribute 'getargspec'

In [46]:
#Исходя из цели автоматической обработки текста, выберем необходимые варианты предобработки текста. 
#Например, для классификации можно воспользоваться нижним регистром, удалением  пунктуации, ссылок, стоп-слов, 
#произвести лемматизацию и токенизацию
##ИЛИ воспользоваться только удалением ссылок. Т.к. это актуальнее для НС, а выбор предобработки сделал для Машинного обученя
df['text_ready']  = df['review'].str.lower()
df['text_ready'] = df['text_ready'].str.replace('\d+', '') 
df['text_ready'] = df['text_ready'].str.replace('[^\w\s]','')
df["text_ready"] = df["text_ready"].apply(stopwords)
df['text_ready'] = df['text_ready'].apply(emoji)
df['text_ready'] = df['text_ready'].apply(remove_urls)
df['text_ready'] = df['text_ready'].apply(html)
df["text_ready"] = df["text_ready"].map(lambda x: " ".join(tokenize_lemmas(x)))
df[['text_ready']].head()

,text_ready
0,качество плохой пошив ужасный горловина напере...
1,товар отдать другой человек получить посылка л...
2,ужасный синтетик тонкий общий представить карт...
3,товар прийти продавец продлить защита мой согл...
4,кофточка голый синтетик носить возможно


In [ ]:
##Пример препобученной модели HuggingFace Torch

In [17]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [23]:
#txt = df.iloc[2,2]
txt = "Хорошая погода! Наконец-то тепло!"
print(txt)

Хорошая погода! Наконец-то тепло!


In [24]:
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [25]:
 predict(txt)

array([1], dtype=int64)